In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
print(torch.__version__)

1.13.1


In [23]:
B, C, H, W = 2, 3, 4 , 4

x = torch.arange(0,B*C*H*W).view(B,C,H,W).float()
x

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]],

         [[32., 33., 34., 35.],
          [36., 37., 38., 39.],
          [40., 41., 42., 43.],
          [44., 45., 46., 47.]]],


        [[[48., 49., 50., 51.],
          [52., 53., 54., 55.],
          [56., 57., 58., 59.],
          [60., 61., 62., 63.]],

         [[64., 65., 66., 67.],
          [68., 69., 70., 71.],
          [72., 73., 74., 75.],
          [76., 77., 78., 79.]],

         [[80., 81., 82., 83.],
          [84., 85., 86., 87.],
          [88., 89., 90., 91.],
          [92., 93., 94., 95.]]]])

In [24]:
batchnorm = nn.BatchNorm2d(C)
batchnorm(x)

tensor([[[[-1.2889, -1.2480, -1.2071, -1.1662],
          [-1.1253, -1.0843, -1.0434, -1.0025],
          [-0.9616, -0.9207, -0.8798, -0.8388],
          [-0.7979, -0.7570, -0.7161, -0.6752]],

         [[-1.2889, -1.2480, -1.2071, -1.1662],
          [-1.1253, -1.0843, -1.0434, -1.0025],
          [-0.9616, -0.9207, -0.8798, -0.8388],
          [-0.7979, -0.7570, -0.7161, -0.6752]],

         [[-1.2889, -1.2480, -1.2071, -1.1662],
          [-1.1253, -1.0843, -1.0434, -1.0025],
          [-0.9616, -0.9207, -0.8798, -0.8388],
          [-0.7979, -0.7570, -0.7161, -0.6752]]],


        [[[ 0.6752,  0.7161,  0.7570,  0.7979],
          [ 0.8388,  0.8798,  0.9207,  0.9616],
          [ 1.0025,  1.0434,  1.0843,  1.1253],
          [ 1.1662,  1.2071,  1.2480,  1.2889]],

         [[ 0.6752,  0.7161,  0.7570,  0.7979],
          [ 0.8388,  0.8798,  0.9207,  0.9616],
          [ 1.0025,  1.0434,  1.0843,  1.1253],
          [ 1.1662,  1.2071,  1.2480,  1.2889]],

         [[ 0.6752,  0.7161,

In [25]:
batchnorm.running_mean, batchnorm.running_var

(tensor([3.1500, 4.7500, 6.3500]), tensor([62.5516, 62.5516, 62.5516]))

In [26]:
batchnorm.weight,batchnorm.bias

(Parameter containing:
 tensor([1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0.], requires_grad=True))

In [27]:
batchnorm.eps

1e-05

In [28]:
x.view(B, C, H* W ).var(dim=-1)

tensor([[22.6667, 22.6667, 22.6667],
        [22.6667, 22.6667, 22.6667]])

In [29]:
x.view(B, C, H* W ).mean(dim=-1)

tensor([[ 7.5000, 23.5000, 39.5000],
        [55.5000, 71.5000, 87.5000]])

In [30]:
x.var(unbiased=True,axis=1)

tensor([[[256., 256., 256., 256.],
         [256., 256., 256., 256.],
         [256., 256., 256., 256.],
         [256., 256., 256., 256.]],

        [[256., 256., 256., 256.],
         [256., 256., 256., 256.],
         [256., 256., 256., 256.],
         [256., 256., 256., 256.]]])

In [31]:
x.mean(axis=1)

tensor([[[16., 17., 18., 19.],
         [20., 21., 22., 23.],
         [24., 25., 26., 27.],
         [28., 29., 30., 31.]],

        [[64., 65., 66., 67.],
         [68., 69., 70., 71.],
         [72., 73., 74., 75.],
         [76., 77., 78., 79.]]])

In [32]:
torch.manual_seed(1)

class MyBatchNorm2d(nn.BatchNorm2d):
    def __init__(self, num_features, eps=1e-5, momentum=0.1,
                 affine=True, track_running_stats=True):
        super(MyBatchNorm2d, self).__init__(
            num_features, eps, momentum, affine, track_running_stats)

    def forward(self, input):
        self._check_input_dim(input)

        exponential_average_factor = 0.0

        if self.training and self.track_running_stats:
            if self.num_batches_tracked is not None:
                self.num_batches_tracked += 1
                if self.momentum is None:  # use cumulative moving average
                    exponential_average_factor = 1.0 / float(self.num_batches_tracked)
                else:  # use exponential moving average
                    exponential_average_factor = self.momentum

        # calculate running estimates
        if self.training:
            mean = input.mean(dim=[0, 2, 3])
            # use biased var in train
            var = input.var([0, 2, 3], unbiased=False)
            n = input.numel() / input.size(1)
            with torch.no_grad():
                self.running_mean = exponential_average_factor * mean + (1 - exponential_average_factor) * self.running_mean
                # update running_var with unbiased var
                self.running_var = exponential_average_factor * var * n / (n - 1) + (1 - exponential_average_factor) * self.running_var
        else:
            mean = self.running_mean
            var = self.running_var

        input = (input - mean[None, :, None, None]) / (torch.sqrt(var[None, :, None, None] + self.eps))
        if self.affine:
            input = input * self.weight[None, :, None, None] + self.bias[None, :, None, None]

        return input
    
norm = nn.BatchNorm2d(3,momentum=0.1)
norm1 = MyBatchNorm2d(3,momentum=0.1)
d = torch.randn(2, 3, 2, 3)

test = norm(d)
test = norm1(d)
norm1.running_var, norm.running_var

(tensor([0.9498, 1.0702, 1.0165]), tensor([0.9498, 1.0702, 1.0165]))

In [33]:
gamma = 0.1
mean = d.mean([0, 2, 3])
var = d.var([0, 2, 3], unbiased=False)
n = d.numel() / d.size(1)
running_variance = gamma * var * n / (n - 1) + ( 1 - gamma) * torch.ones_like(var)
running_variance

tensor([0.9498, 1.0702, 1.0165])

In [34]:
test = norm(d)

norm.running_var

tensor([0.9046, 1.1333, 1.0313])

In [35]:
norm1.running_var

tensor([0.9498, 1.0702, 1.0165])

In [36]:
"""
Comparison of manual BatchNorm2d layer implementation in Python and
nn.BatchNorm2d

@author: ptrblck
"""

import torch
import torch.nn as nn


def compare_bn(bn1, bn2):
    err = False
    if not torch.allclose(bn1.running_mean, bn2.running_mean):
        print('Diff in running_mean: {} vs {}'.format(
            bn1.running_mean, bn2.running_mean))
        err = True

    if not torch.allclose(bn1.running_var, bn2.running_var):
        print('Diff in running_var: {} vs {}'.format(
            bn1.running_var, bn2.running_var))
        err = True

    if bn1.affine and bn2.affine:
        if not torch.allclose(bn1.weight, bn2.weight):
            print('Diff in weight: {} vs {}'.format(
                bn1.weight, bn2.weight))
            err = True

        if not torch.allclose(bn1.bias, bn2.bias):
            print('Diff in bias: {} vs {}'.format(
                bn1.bias, bn2.bias))
            err = True

    if not err:
        print('All parameters are equal!')




# Init BatchNorm layers
my_bn = MyBatchNorm2d(3, affine=True)
bn = nn.BatchNorm2d(3, affine=True)

compare_bn(my_bn, bn)  # weight and bias should be different
# Load weight and bias
my_bn.load_state_dict(bn.state_dict())
compare_bn(my_bn, bn)

# Run train
for _ in range(10):
    scale = torch.randint(1, 10, (1,)).float()
    bias = torch.randint(-10, 10, (1,)).float()
    x = torch.randn(10, 3, 100, 100) * scale + bias
    out1 = my_bn(x)
    out2 = bn(x)
    print(my_bn.running_var,bn.running_var)
    compare_bn(my_bn, bn)

    torch.allclose(out1, out2)
    print('Max diff: ', (out1 - out2).abs().max())

# Run eval
my_bn.eval()
bn.eval()
for _ in range(10):
    scale = torch.randint(1, 10, (1,)).float()
    bias = torch.randint(-10, 10, (1,)).float()
    x = torch.randn(10, 3, 100, 100) * scale + bias
    out1 = my_bn(x)
    out2 = bn(x)
    compare_bn(my_bn, bn)

    torch.allclose(out1, out2)
    print('Max diff: ', (out1 - out2).abs().max())

All parameters are equal!
All parameters are equal!
tensor([3.3909, 3.4120, 3.3850]) tensor([3.3909, 3.4120, 3.3850])
All parameters are equal!
Max diff:  tensor(9.5367e-07, grad_fn=<MaxBackward1>)
tensor([3.4507, 3.4710, 3.4473]) tensor([3.4507, 3.4710, 3.4473])
All parameters are equal!
Max diff:  tensor(4.7684e-07, grad_fn=<MaxBackward1>)
tensor([6.6752, 6.7449, 6.6850]) tensor([6.6752, 6.7449, 6.6850])
All parameters are equal!
Max diff:  tensor(4.7684e-07, grad_fn=<MaxBackward1>)
tensor([9.6064, 9.6532, 9.6202]) tensor([9.6064, 9.6532, 9.6202])
All parameters are equal!
Max diff:  tensor(4.7684e-07, grad_fn=<MaxBackward1>)
tensor([10.2415, 10.2822, 10.2644]) tensor([10.2415, 10.2822, 10.2644])
All parameters are equal!
Max diff:  tensor(7.1526e-07, grad_fn=<MaxBackward1>)
tensor([15.6107, 15.6859, 15.6040]) tensor([15.6107, 15.6859, 15.6040])
All parameters are equal!
Max diff:  tensor(7.1526e-07, grad_fn=<MaxBackward1>)
tensor([20.4277, 20.5349, 20.4101]) tensor([20.4277, 20.5349